This notebook uses Sympy to expand the equation of motion of the $Q$-tensor in terms of its degrees of freedom.
Given this, residuals corresponding to different time-discretization techniques may be calculated.
With the residual in hand, Jacobians may also be easily calculated by using the Gateaux derivative.
Finally, we may generate the corresponding deal.II assembly code with the code generation module and some custom formatting.

In [1]:
from sympy import *

## Generate necessary symbols

In [21]:
def gen_functions(formattable_string, n, *coords):
    
    return [Function(formattable_string.format(i), real=True)(*coords) for i in range(n)]  

In [71]:
vec_dim = 5
mat_dim = 3

x, y, z = symbols('x y z', real=True)
xi = Matrix([x, y, z])

Q_vec = Matrix(gen_functions('Q_{}', vec_dim, x, y))
Lambda_vec = Matrix(gen_functions(r'\Lambda_{}', vec_dim, Q_vec[0], Q_vec[1], Q_vec[2], Q_vec[3], Q_vec[4]))
phi_i = Function('\phi_i', real=True)(x, y)
phi_j = Function('\phi_j', real=True)(x, y)

alpha, L2, L3 = symbols('alpha L_2 L_3', real=True)

In [72]:
Q_mat = Matrix([[Q_vec[0], Q_vec[1], Q_vec[2]],
                [Q_vec[1], Q_vec[3], Q_vec[4]],
                [Q_vec[2], Q_vec[4], -(Q_vec[0] + Q_vec[3])]])

Lambda_mat = Matrix([[Lambda_vec[0], Lambda_vec[1], Lambda_vec[2]],
                     [Lambda_vec[1], Lambda_vec[3], Lambda_vec[4]],
                     [Lambda_vec[2], Lambda_vec[4], -(Lambda_vec[0] + Lambda_vec[3])]])

In [73]:
basis = []
basis.append(Matrix([[1, 0, 0],
                     [0, 0, 0],
                     [0, 0, -1]]))
basis.append(Matrix([[0, 1, 0],
                     [1, 0, 0],
                     [0, 0, 0]]))
basis.append(Matrix([[0, 0, 1],
                     [0, 0, 0],
                     [1, 0, 0]]))
basis.append(Matrix([[0, 0, 0],
                     [0, 1, 0],
                     [0, 0, -1]]))
basis.append(Matrix([[0, 0, 0],
                     [0, 0, 1],
                     [0, 1, 0]]))

In [74]:
Phi_i = []
for basis_vec in basis:
    Phi_i.append(phi_i * basis_vec)
    
Phi_j = []
for basis_vec in basis:
    Phi_j.append(phi_j * basis_vec)

## Write weak form

In [75]:
T = []

for m in range(vec_dim):
    T.append(
        [alpha * sum(Phi_i[m][i, j] * Q_mat[i, j] 
                     for i in range(3) for j in range(3)),
         - sum(Phi_i[m][i, j] * Lambda_mat[i, j] 
               for i in range(3) for j in range(3)),
         - L3 / 2 * sum(Phi_i[m][i, j] * Q_mat[k, l].diff(xi[i]) * Q_mat[k, l].diff(xi[j])
                        for i in range(3) for j in range(3) for k in range(3) for l in range(3)),
         - sum(Phi_i[m][i, j].diff(xi[k]) * Q_mat[i, j].diff(xi[k])
               for i in range(3) for j in range(3) for k in range(3)),
         - L2 * sum(Phi_i[m][i, j].diff(xi[k]) * Q_mat[k, i].diff(xi[j])
                    for i in range(3) for j in range(3) for k in range(3)),
         - L3 * sum(Phi_i[m][i, j].diff(xi[k]) * Q_mat[k, l] * Q_mat[i, j].diff(xi[l])
                    for i in range(3) for j in range(3) for k in range(3) for l in range(3))]
    )

In [76]:
# i is test function basis vector, j is which term in the weak form
for i in range(vec_dim):
    for j in range(6):
        T[i][j] = simplify(T[i][j])
        display(T[i][j])

alpha*(2*Q_0(x, y) + Q_3(x, y))*\phi_i(x, y)

(-2*\Lambda_0(Q_0(x, y), Q_1(x, y), Q_2(x, y), Q_3(x, y), Q_4(x, y)) - \Lambda_3(Q_0(x, y), Q_1(x, y), Q_2(x, y), Q_3(x, y), Q_4(x, y)))*\phi_i(x, y)

L_3*(-Derivative(Q_0(x, y), x)**2 - Derivative(Q_0(x, y), x)*Derivative(Q_3(x, y), x) - Derivative(Q_1(x, y), x)**2 - Derivative(Q_2(x, y), x)**2 - Derivative(Q_3(x, y), x)**2 - Derivative(Q_4(x, y), x)**2)*\phi_i(x, y)

-2*Derivative(Q_0(x, y), x)*Derivative(\phi_i(x, y), x) - 2*Derivative(Q_0(x, y), y)*Derivative(\phi_i(x, y), y) - Derivative(Q_3(x, y), x)*Derivative(\phi_i(x, y), x) - Derivative(Q_3(x, y), y)*Derivative(\phi_i(x, y), y)

-L_2*(Derivative(Q_0(x, y), x)*Derivative(\phi_i(x, y), x) + Derivative(Q_1(x, y), x)*Derivative(\phi_i(x, y), y))

...

-2*\Lambda_4(Q_0(x, y), Q_1(x, y), Q_2(x, y), Q_3(x, y), Q_4(x, y))*\phi_i(x, y)

0

-2*Derivative(Q_4(x, y), x)*Derivative(\phi_i(x, y), x) - 2*Derivative(Q_4(x, y), y)*Derivative(\phi_i(x, y), y)

-L_2*(Derivative(Q_2(x, y), y)*Derivative(\phi_i(x, y), x) + Derivative(Q_4(x, y), y)*Derivative(\phi_i(x, y), y))

-2*L_3*(Q_0(x, y)*Derivative(Q_4(x, y), x)*Derivative(\phi_i(x, y), x) + Q_1(x, y)*Derivative(Q_4(x, y), x)*Derivative(\phi_i(x, y), y) + Q_1(x, y)*Derivative(Q_4(x, y), y)*Derivative(\phi_i(x, y), x) + Q_3(x, y)*Derivative(Q_4(x, y), y)*Derivative(\phi_i(x, y), y))

# Write weak form Jacobian

In [83]:
dQ_vec = Matrix(gen_functions(r'\delta\ Q_{}', vec_dim, x, y))
tau = symbols('tau', real=True)

In [84]:
Q_subs = {Q_vec[i]: Q_vec[i] + tau * dQ_vec[i] for i in range(vec_dim)}

In [92]:
T[1][3].subs(Q_subs).diff(tau).subs(tau, 0)

-2*Derivative(\delta\ Q_1(x, y), x)*Derivative(\phi_i(x, y), x) - 2*Derivative(\delta\ Q_1(x, y), y)*Derivative(\phi_i(x, y), y)